In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

class BasketImpliedCorrelation:
    """
    BASKET IMPLIED CORRELATION CALCULATOR
    =====================================
    
    PURPOSE: Calculate the implied correlation for a custom basket of stocks
    OUTPUT: Daily implied correlation values that can be used for:
             - Dispersion trading signals
             - Correlation risk monitoring  
             - Volatility strategy development
             - Risk management
    
    METHOD: Uses the relationship between index volatility and component volatilities
            Derived from portfolio variance formula:
            σ_index² = Σw_i²σ_i² + ρ_implied × ((Σw_iσ_i)² - Σw_i²σ_i²)
    """
    
    def __init__(self):
        self.implied_vol_data = None
        self.weights = None
        self.realized_corr_matrices = None
        self.vix_data = None
        
    def load_data(self, file_path):
        """Load all required data"""
        logger.info("Loading data for basket implied correlation...")
        
        # 1. Load implied volatility data (INCLUDING TESLA)
        self._load_implied_vol_data(file_path)
        
        # 2. Load and normalize weights (INCLUDING TESLA)
        self._initialize_weights()
        
        # 3. Load VIX data
        self._load_vix_data(file_path)
        
        # 4. Load realized correlation matrices
        self._load_realized_correlations()
        
        logger.info("All data loaded successfully")
        
    def _load_implied_vol_data(self, file_path):
        """Load implied volatility data - INCLUDING TESLA"""
        iv_sheet = pd.read_excel(file_path, sheet_name='implied_vol_30d', header=1)
        self.implied_vol_data = pd.DataFrame()
        
        dates_column = iv_sheet.iloc[1:, 0]
        self.implied_vol_data['Date'] = pd.to_datetime(dates_column, format='%m/%d/%Y', errors='coerce')
        
        # INCLUDING TESLA (Column 8)
        stock_columns = {
            'AAPL': 1, 'MSFT': 2, 'NVDA': 3, 'AMZN': 4, 'META': 5,
            'GOOGL': 6, 'GOOG': 7, 'TSLA': 8, 'BRK.B': 9, 'LLY': 10
        }
        
        for ticker, col_idx in stock_columns.items():
            self.implied_vol_data[ticker] = pd.to_numeric(iv_sheet.iloc[1:, col_idx], errors='coerce')
        
        self.implied_vol_data = self.implied_vol_data.dropna(subset=['Date'])
        self.implied_vol_data = self.implied_vol_data.set_index('Date').sort_index()
        
        logger.info(f"Loaded implied vol data: {len(self.implied_vol_data)} records")
        logger.info(f"Stocks included: {list(stock_columns.keys())}")
        
    def _initialize_weights(self):
        """Initialize and normalize weights - INCLUDING TESLA"""
        # INCLUDING TESLA at 2.37%
        raw_weights = {
            'NVDA': 0.0738, 'AAPL': 0.0663, 'MSFT': 0.0613, 'AMZN': 0.0427,
            'GOOGL': 0.0288, 'GOOG': 0.0270, 'META': 0.0255, 'TSLA': 0.0237,
            'BRK.B': 0.0179, 'LLY': 0.0136
        }
        
        total_weight = sum(raw_weights.values())
        self.weights = {ticker: weight/total_weight for ticker, weight in raw_weights.items()}
        
        logger.info(f"Total basket weight: {total_weight:.1%} of SPX")
        logger.info(f"Normalized weight sum: {sum(self.weights.values()):.4f}")
        
    def _load_vix_data(self, file_path):
        """Load VIX data"""
        vix_sheet = pd.read_excel(file_path, sheet_name='VIX_VVIX_VXC1-6', header=2)
        self.vix_data = pd.DataFrame({
            'Date': vix_sheet.iloc[:, 0],
            'VIX': vix_sheet.iloc[:, 3]
        })
        self.vix_data['Date'] = pd.to_datetime(self.vix_data['Date'], errors='coerce')
        self.vix_data = self.vix_data.dropna(subset=['Date', 'VIX'])
        self.vix_data = self.vix_data.set_index('Date').sort_index()
        
        logger.info(f"Loaded VIX data: {len(self.vix_data)} records")
        
    def _load_realized_correlations(self):
        """Load realized correlation matrices"""
        try:
            corr_file = 'rolling_corr_matrix_30D.csv'
            if not os.path.exists(corr_file):
                logger.warning(f"Correlation file not found: {corr_file}")
                self.realized_corr_matrices = None
                return
                
            corr_data = pd.read_csv(corr_file)
            corr_data.columns = corr_data.columns.str.strip()
            corr_data['Date'] = pd.to_datetime(corr_data['Date'])
            
            data_column = corr_data.columns[1]
            corr_data[data_column] = corr_data[data_column].str.strip()
            
            unique_dates = corr_data['Date'].unique()
            self.realized_corr_matrices = {}
            
            # Clean ticker names
            ticker_columns = [col for col in corr_data.columns if col not in ['Date', data_column]]
            clean_ticker_map = {}
            for col in ticker_columns:
                clean_name = col.replace(' US Equity', '').strip()
                clean_ticker_map[col] = clean_name
            
            # Process each date's correlation matrix
            for date in unique_dates:
                date_data = corr_data[corr_data['Date'] == date].copy()
                date_data = date_data.set_index(data_column)
                date_data = date_data.drop(columns=['Date'])
                date_data = date_data.rename(columns=clean_ticker_map)
                date_data.index = date_data.index.str.replace(' US Equity', '').str.strip()
                
                # Align with our portfolio tickers (INCLUDING TESLA)
                our_tickers = list(self.weights.keys())
                available_tickers = [t for t in our_tickers if t in date_data.index and t in date_data.columns]
                
                if len(available_tickers) < 2:
                    continue
                
                corr_matrix = date_data.loc[available_tickers, available_tickers]
                self.realized_corr_matrices[date] = corr_matrix
                
            logger.info(f"Loaded correlation matrices: {len(self.realized_corr_matrices)} dates")
            logger.info(f"Available tickers: {len(available_tickers)}/{len(self.weights)} stocks")
            
        except Exception as e:
            logger.error(f"Error loading correlation matrices: {e}")
            self.realized_corr_matrices = None
    
    def calculate_implied_correlation(self):
        """
        CALCULATE IMPLIED CORRELATION FOR THE BASKET
        ============================================
        
        Returns: Daily implied correlation values for your custom basket
        
        This is the CORE output that can be used for:
        - Building dispersion trading strategies
        - Correlation risk analysis  
        - Volatility forecasting
        - Signal generation
        
        Formula: ρ_implied = (VIX² - Σw_i²σ_i²) / ((Σw_iσ_i)² - Σw_i²σ_i²)
        """
        logger.info("Calculating basket implied correlation...")
        
        results = []
        
        for date in self.implied_vol_data.index:
            try:
                if date not in self.vix_data.index:
                    continue
                    
                # Get current implied vols for all stocks (INCLUDING TESLA)
                ivs = {ticker: self.implied_vol_data.loc[date, ticker] / 100 
                       for ticker in self.weights.keys()}
                
                vix_val = self.vix_data.loc[date, 'VIX'] / 100
                
                # Calculate implied correlation using the portfolio variance formula
                implied_corr = self._calculate_single_implied_correlation(vix_val, ivs, self.weights)
                
                # Also calculate the average realized correlation for comparison
                avg_realized_corr = self._get_average_realized_correlation(date)
                
                results.append({
                    'Date': date,
                    'Implied_Correlation': implied_corr,
                    'Avg_Realized_Correlation': avg_realized_corr,
                    'VIX': vix_val * 100,  # Convert back to percentage
                    'Basket_Weighted_IV': np.sum([self.weights[t] * ivs[t] for t in self.weights.keys()]) * 100
                })
                
            except Exception as e:
                logger.debug(f"Skipping {date}: {e}")
                continue
        
        results_df = pd.DataFrame(results).set_index('Date')
        logger.info(f"Calculated {len(results_df)} implied correlation values")
        
        return results_df
    
    def _calculate_single_implied_correlation(self, vix_val, ivs_dict, weights_dict):
        """
        Calculate implied correlation for a single date
        
        Formula: ρ_implied = (VIX² - A) / B
        Where:
          A = Σw_i²σ_i²      (individual variance terms)
          B = (Σw_iσ_i)² - A (cross-term variance)
        """
        vix_var = vix_val ** 2
        
        # Calculate individual variance component
        A = sum(weights_dict[t]**2 * ivs_dict[t]**2 for t in weights_dict.keys())
        
        # Calculate cross-term component
        weighted_iv_sum = sum(weights_dict[t] * ivs_dict[t] for t in weights_dict.keys())
        B = weighted_iv_sum**2 - A
        
        if abs(B) < 1e-10:
            return np.nan
        
        implied_corr = (vix_var - A) / B
        
        # Bound between 0 and 1 (theoretically)
        return max(0.0, min(1.0, implied_corr))
    
    def _get_average_realized_correlation(self, date):
        """Get average realized correlation for comparison"""
        if self.realized_corr_matrices and date in self.realized_corr_matrices:
            corr_matrix = self.realized_corr_matrices[date]
            # Exclude diagonal (1.0 values)
            mask = ~np.eye(corr_matrix.shape[0], dtype=bool)
            return corr_matrix.values[mask].mean()
        else:
            return np.nan

# ============================================================================
# MAIN EXECUTION - BASKET IMPLIED CORRELATION
# ============================================================================

def main():
    """
    BASKET IMPLIED CORRELATION CALCULATION
    ======================================
    
    This script calculates the daily implied correlation for your custom basket
    of 10 SPX constituents (including Tesla).
    
    OUTPUT: Clean DataFrame with implied correlation values that can be used for:
            - Dispersion strategy development
            - Correlation risk analysis
            - Signal generation
            - Risk management
    """
    try:
        # Setup
        os.chdir(os.path.expanduser('~/Downloads'))
        file_path = 'merged file (2).xlsx'
        
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Data file not found: {file_path}")
        
        # Initialize calculator
        calculator = BasketImpliedCorrelation()
        calculator.load_data(file_path)
        
        # Calculate implied correlation
        implied_corr_results = calculator.calculate_implied_correlation()
        
        # Display results
        print("\n" + "="*80)
        print("BASKET IMPLIED CORRELATION RESULTS")
        print("="*80)
        
        print(f"\nSummary Statistics:")
        print(implied_corr_results.describe())
        
        print(f"\nKey Metrics:")
        print(f"  Average Implied Correlation: {implied_corr_results['Implied_Correlation'].mean():.3f}")
        print(f"  Implied Correlation Std Dev: {implied_corr_results['Implied_Correlation'].std():.3f}")
        print(f"  Implied Correlation Range:   {implied_corr_results['Implied_Correlation'].min():.3f} to {implied_corr_results['Implied_Correlation'].max():.3f}")
        
        if not implied_corr_results['Avg_Realized_Correlation'].isna().all():
            print(f"  Average Realized Correlation: {implied_corr_results['Avg_Realized_Correlation'].mean():.3f}")
        
        print(f"\nInterpretation:")
        print(f"  - Implied correlation represents the market's expectation of future co-movement")
        print(f"  - Values typically range between 0.3-0.7 for diversified baskets")
        print(f"  - Higher values suggest expected high correlation (diversification benefits low)")
        print(f"  - Lower values suggest expected low correlation (diversification benefits high)")
        
        # Save clean results for further analysis
        output_file = 'basket_implied_correlation.csv'
        
        # Create a clean output with just the essential columns
        clean_output = implied_corr_results[['Implied_Correlation', 'Avg_Realized_Correlation', 'VIX']].copy()
        clean_output.to_csv(output_file)
        
        logger.info(f"Implied correlation results saved to '{output_file}'")
        
        # Display sample of results
        print(f"\nSample Results (First 10 days):")
        print(clean_output.head(10).to_string(float_format="%.3f"))
        
        print(f"\nBasket implied correlation calculation complete!")
        print(f"You can now use these implied correlation values to:")
        print(f"   - Build dispersion trading strategies")
        print(f"   - Analyze correlation risk premium") 
        print(f"   - Develop volatility forecasting models")
        print(f"   - Create mean-reversion signals")
        
    except Exception as e:
        logger.error(f"Calculation failed: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()